In [1]:
import pandas as pd

data_source = "/home/fra/DataMart/datacentre/olddata/"
train_file = "adult_train.csv"
test_file = "adult_test.csv"

# load data
df = pd.read_csv(data_source + train_file)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
import sys
mylib = '/home/fra/Project/pyProj/mlib'

In [3]:
sys.path.append(mylib)

In [18]:
import utility as util
import explore as ex
import pipeline as pl
import model as ml
import transform as trf
from imp import reload

In [7]:
exclusions=['DATA_DT', 'PROCESS_DTTM', 'GCIS_KEY']
target_col = 'INCOME'

In [5]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [8]:
stp = pl.StandardPipeline(target_col, exclusions)
stp.process(df, ntransformer=StandardScaler, ctransformer=OrdinalEncoder)

In [12]:
(X, y) = stp.out()
features = stp.features

### basic classifier

In [15]:
from sklearn.model_selection import train_test_split
features = df.columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 

### XGB

In [16]:
xgb = XGBClassifier().fit(X_train, y_train)

In [19]:
ev = ml.BinaryClassificationEvaluator(xgb)

In [21]:
ev.performance_summary(X_test, y_test)

{'Accuracy': 0.8661070733954346,
 'Consfusion Matrix': array([[6921,  500],
        [ 808, 1540]]),
 'AUC': 0.9190636544802299,
 'Gini': 0.8381273089604597}

### single classifiers

In [22]:
nb = GaussianNB().fit(X_train, y_train)
knn = KNeighborsClassifier().fit(X_train, y_train)
dtc = DecisionTreeClassifier().fit(X_train, y_train)

In [23]:
nbev = ml.BinaryClassificationEvaluator(nb)
knnev = ml.BinaryClassificationEvaluator(knn)
dtcev = ml.BinaryClassificationEvaluator(dtc)

In [24]:
nbev.performance_summary(X_test, y_test)

{'Accuracy': 0.8032551950046064,
 'Consfusion Matrix': array([[7062,  359],
        [1563,  785]]),
 'AUC': 0.8539114562086918,
 'Gini': 0.7078229124173836}

In [25]:
knnev.performance_summary(X_test, y_test)

{'Accuracy': 0.8247517657897431,
 'Consfusion Matrix': array([[6736,  685],
        [1027, 1321]]),
 'AUC': 0.8406396611026263,
 'Gini': 0.6812793222052527}

In [26]:
dtcev.performance_summary(X_test, y_test)

{'Accuracy': 0.8072474152932746,
 'Consfusion Matrix': array([[6439,  982],
        [ 901, 1447]]),
 'AUC': 0.7419709928108157,
 'Gini': 0.4839419856216314}

### Bagging

In [27]:
from sklearn.ensemble import BaggingClassifier

In [33]:
bnb = BaggingClassifier(GaussianNB()).fit(X_train, y_train)
bknn = BaggingClassifier(KNeighborsClassifier()).fit(X_train, y_train)
bdtc = BaggingClassifier(DecisionTreeClassifier()).fit(X_train, y_train)

In [35]:
bnv_ev = ml.BinaryClassificationEvaluator(bnb)

In [36]:
bnv_ev.performance_summary(X_test, y_test)

{'Accuracy': 0.8031528303818201,
 'Consfusion Matrix': array([[7060,  361],
        [1562,  786]]),
 'AUC': 0.8537524847186504,
 'Gini': 0.7075049694373008}

In [37]:
ml.BinaryClassificationEvaluator(bknn).performance_summary(X_test, y_test)

{'Accuracy': 0.8223973794656567,
 'Consfusion Matrix': array([[6747,  674],
        [1061, 1287]]),
 'AUC': 0.8555596806521022,
 'Gini': 0.7111193613042044}

In [38]:
ml.BinaryClassificationEvaluator(bdtc).performance_summary(X_test, y_test)

{'Accuracy': 0.8447128672330843,
 'Consfusion Matrix': array([[6906,  515],
        [1002, 1346]]),
 'AUC': 0.8730503610202367,
 'Gini': 0.7461007220404734}

### Stacking

In [40]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [44]:
lr = LogisticRegression()
estimators = [('nb', nb), ('knn', knn), ('dtc', dtc)]

In [45]:
sclf = StackingClassifier(estimators=estimators, final_estimator=lr )
sclf.fit(X_train, y_train)

StackingClassifier(cv=None,
                   estimators=[('nb',
                                GaussianNB(priors=None, var_smoothing=1e-09)),
                               ('knn',
                                KNeighborsClassifier(algorithm='auto',
                                                     leaf_size=30,
                                                     metric='minkowski',
                                                     metric_params=None,
                                                     n_jobs=None, n_neighbors=5,
                                                     p=2, weights='uniform')),
                               ('dtc',
                                DecisionTreeClassifier(ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                             

In [46]:
ml.BinaryClassificationEvaluator(sclf).performance_summary(X_test, y_test)

{'Accuracy': 0.8349882280683796,
 'Consfusion Matrix': array([[6935,  486],
        [1126, 1222]]),
 'AUC': 0.8825959103120731,
 'Gini': 0.7651918206241461}

### stacking with XGB

In [47]:
lr = LogisticRegression()
estimators = [('nb', nb), ('knn', knn), ('dtc', dtc), ('xgb', xgb)]

In [49]:
xgclf = StackingClassifier(estimators=estimators, final_estimator=lr )
xgclf.fit(X_train, y_train)

StackingClassifier(cv=None,
                   estimators=[('nb',
                                GaussianNB(priors=None, var_smoothing=1e-09)),
                               ('knn',
                                KNeighborsClassifier(algorithm='auto',
                                                     leaf_size=30,
                                                     metric='minkowski',
                                                     metric_params=None,
                                                     n_jobs=None, n_neighbors=5,
                                                     p=2, weights='uniform')),
                               ('dtc',
                                DecisionTreeClassifier(ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                             

In [50]:
ml.BinaryClassificationEvaluator(xgclf).performance_summary(X_test, y_test)

{'Accuracy': 0.8691780120790255,
 'Consfusion Matrix': array([[6987,  434],
        [ 844, 1504]]),
 'AUC': 0.9179949872903155,
 'Gini': 0.835989974580631}